# This is an example of opening multiple .root file using uproot4, dask and displaying data into a single aggregated histogram

## 1. Importing all the necessary packages

In [2]:
import sys
sys.path.insert(0, '..')
# sys.path.insert(1, '../python/')
# import root_numpy as rnp
import uproot4
import python.file_manager as fm
import ROOT

Welcome to JupyROOT 6.26/11


## 2. Importing the second batch of packages

In [5]:
import numpy as np
import fsspec
import distributed
import awkward as ak
import dask_awkward as dak
print("Loaded")

Loaded


## 3. Importing histogram libraries

In [6]:
import hist

import dask_histogram as dh
import boost_histogram as bh
print("Loaded")

/data/gsvedas/.virtualenvs/gintautas/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/data/gsvedas/.virtualenvs/gintautas/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/data/gsvedas/.virtualenvs/gintautas/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/data/gsvedas/.virtualenvs/gintautas/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


Loaded


## 4. Opening a distributed client for Dask

In [7]:
client = distributed.Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 48,Total memory: 62.29 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40345,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 48
Started: Just now,Total memory: 62.29 GiB
Comm: tcp://127.0.0.1:45141,Total threads: 6
Dashboard: http://127.0.0.1:39011/status,Memory: 7.79 GiB
Nanny: tcp://127.0.0.1:33407,


## 7. Code from ntuple-analysis package, manipulating a Tree object

In [8]:
import pandas as pd
import datetime
import resource
import gc
import awkward as ak
import awkward_pandas as akpd
import vector
vector.register_awkward()

class TreeReader(object):
    def __init__(self, entry_range, max_events):
        self.tree = None
        self._branches = []
        # this is the gloabl "entry" across files
        self.global_entry = -1
        # this is the "entry" local to the open file (reset to 0) every new file
        self.file_entry = -1
        self.max_events = max_events
        self.entry_range = entry_range

        self.n_tot_entries = 0

    def setTree(self, uptree):
        self.tree = uptree
        self._branches = []
        branch_blacklist = ['tc_wafer',
                            'tc_cell',
                            'tc_waferu',
                            'tc_waferv',
                            'tc_cellu',
                            'tc_cellv',
                            'gen_PUNumInt',
                            'gen_TrueNumInt',
                            # 'gen_daughters', 
                            'simpart_posx', 'simpart_posy', 'simpart_posz',
                            ]
        if len(self._branches) == 0:
            self._branches = [br for br in self.tree.keys() if br not in branch_blacklist]
        print(f'open new tree file with # entries: {self.tree.num_entries}')
        self.file_entry = -1

    def next(self, debug=0):

        if self.max_events != -1:
            if self.n_tot_entries == self.max_events:
                print('END loop for max_event!')
                # we processed the max # of events
                return False
        if self.entry_range[1] != -1:
            if self.global_entry == self.entry_range[1]:
                print('END loop for entry_range')
                return False
        if self.file_entry == self.tree.num_entries-1:
            print('END loop for end_of_file')
            return False

        if self.global_entry == -1:
            self.global_entry = self.entry_range[0]
            self.file_entry = self.entry_range[0]
        else:
            self.file_entry += 1
            self.global_entry += 1

        # entry is the cursor in the file: when we open a new one (not the first) needs to be set to 0 again
        if debug >= 2 or self.global_entry % 1000 == 0:
            self.printEntry()
        
        self.n_tot_entries += 1
        return True

    def printEntry(self):
        print("--- File entry: {}, global entry: {}, tot # events: {} @ {}, MaxRSS {:.2f} Mb".format(
            self.file_entry,
            self.global_entry,
            self.n_tot_entries,
            datetime.datetime.now(),
            resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1E6))
        # print(self.tree.keys())
        # print(self.tree.arrays(['run', 'lumi', 'event'], library='pd', entry_start=self.file_entry, entry_stop=self.file_entry+1))
        # self.dump_garbage()
    
    def dump_garbage(self):
        """
        show us what's the garbage about
        """
            
        # force collection
        print ("\nGARBAGE:")
        gc.collect()

        print ("\nGARBAGE OBJECTS:")
        for x in gc.garbage:
            s = str(x)
            if len(s) > 80: s = s[:80]
            print (type(x),"\n  ", s)


    def getDataFrame(self, prefix, entry_block, fallback=None):
        branches = [br for br in self._branches
                    if br.startswith(prefix+'_') and
                    not br == '{}_n'.format(prefix)]
        names = ['_'.join(br.split('_')[1:]) for br in branches]
        name_map = dict(zip(names, branches))
        if len(branches) == 0:
            if fallback is not None:
                return self.getDataFrame(prefix=fallback, entry_block=entry_block)
            prefs = set([br.split('_')[0] for br in self._branches])
            print(f'stored branch prefixes are: {prefs}')
            raise ValueError(f'[TreeReader::getDataFrame] No branches with prefix: {prefix}')
        
        akarray = self.tree.arrays(names, 
                                   library='ak', 
                                   aliases=name_map, 
                                   entry_start=self.file_entry, 
                                   entry_stop=self.file_entry+entry_block)
        
        print("TYPE AKARRAY")
        print(akarray)
        records = {}
        for field in akarray.fields:
            records[field] = akarray[field]
        
        if 'pt' in names and 'eta' in names and 'phi' in names:
            if not 'mass' in names and not 'energy' in names:
                records['mass'] = 0.*akarray['pt']
            return vector.zip(records)

        return ak.zip(records)

        # FIXME: we should probably do an ak.Record using sometjhing along the lines of:
        # ele_rec = ak.zip({'pt': tkele.pt, 'eta': tkele.eta, 'phi': tkele.phi}, with_name="pippo")
        # this would allow to handle the records and assign behaviours....

        # return akarray

In [9]:
def getUpTree(uprobj, name):
    parts = name.split('/')
    # if len(parts) > 1:
    #     return getUpTree(uprobj, '/'.join(parts[1:]))
    return uprobj[name]

## 7.1 Opening a .ROOT file, reading a uproot.models.TTree.Model_TTree_v20

In [10]:
files_with_protocol = ['root://eoscms.cern.ch//eos/cms/store/cmst3/group/l1tr/cerminar/l1teg/fp_ntuples/DoubleElectron_FlatPt-1To100_PU200/FP/v100D/perfNano_8830991_1.root']
range_ev = (0, 1)
tree_reader = TreeReader(range_ev, 4)

tree_file = uproot4.open(files_with_protocol[0], num_workers=1)
ttree = getUpTree(tree_file, 'Events')

In [22]:
type(ttree)

uproot.models.TTree.Model_TTree_v20

### 7.2 Creating a dictionary mapping to rename branches

In [11]:
branches = [br for br in ttree.keys()
            if br.startswith('TkEleL2'+'_') and
            not br == '{}_n'.format('TkEleL2')]
names = ['_'.join(br.split('_')[1:]) for br in branches]

name_map = dict(zip(names, branches))

In [26]:
name_map

{'eta': 'TkEleL2_eta',
 'pfIso': 'TkEleL2_pfIso',
 'pfIsoPV': 'TkEleL2_pfIsoPV',
 'phi': 'TkEleL2_phi',
 'pt': 'TkEleL2_pt',
 'puppiIso': 'TkEleL2_puppiIso',
 'puppiIsoPV': 'TkEleL2_puppiIsoPV',
 'tkEta': 'TkEleL2_tkEta',
 'tkIso': 'TkEleL2_tkIso',
 'tkIsoPV': 'TkEleL2_tkIsoPV',
 'tkPhi': 'TkEleL2_tkPhi',
 'tkPt': 'TkEleL2_tkPt',
 'vz': 'TkEleL2_vz',
 'charge': 'TkEleL2_charge',
 'hwQual': 'TkEleL2_hwQual'}

### 7.2.1 Reversed mapping

In [24]:
name_map_new = dict(zip(branches, names))

In [25]:
name_map_new

{'TkEleL2_eta': 'eta',
 'TkEleL2_pfIso': 'pfIso',
 'TkEleL2_pfIsoPV': 'pfIsoPV',
 'TkEleL2_phi': 'phi',
 'TkEleL2_pt': 'pt',
 'TkEleL2_puppiIso': 'puppiIso',
 'TkEleL2_puppiIsoPV': 'puppiIsoPV',
 'TkEleL2_tkEta': 'tkEta',
 'TkEleL2_tkIso': 'tkIso',
 'TkEleL2_tkIsoPV': 'tkIsoPV',
 'TkEleL2_tkPhi': 'tkPhi',
 'TkEleL2_tkPt': 'tkPt',
 'TkEleL2_vz': 'vz',
 'TkEleL2_charge': 'charge',
 'TkEleL2_hwQual': 'hwQual'}

### (old version) 7.3 reading contents of the ttree object via .arrays function

In [ ]:
akarray = ttree.arrays(names, 
                           library='ak', 
                           aliases=name_map, 
                           entry_start=0, 
                           entry_stop=1001)

### (new version) 7.3 reading contents of the ttree lazily, via uproot4.dask

In [14]:
akarray_new = uproot4.dask(ttree, library='ak')

### 7.4 Renaming columns

In [15]:
import dask.dataframe as dd

In [17]:
df = dak.to_dataframe(akarray_new)
df = df.rename(columns=name_map_new)

### 7.4.2 Converting dask dataframe to dask array

In [18]:
dask_array_simple = df.to_dask_array()

In [19]:
dask_array_simple

dask.array<values, shape=(nan, 153), dtype=float64, chunksize=(nan, 153), chunktype=numpy.ndarray>

### 7.4.3 Converting dask array to dask awkward array

In [21]:
dak_array = dak.from_dask_array(dask_array_simple)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.